# Лабораторная работа No 9
## Алгоритмы кластерного анализа данных

In [41]:
import json
import numpy as np

from collections import Counter


DATASET = 'data_546.json'

Исследуем полученные данные

In [29]:
with open(DATASET, 'r') as file:
    data = json.load(file)

# Получаем необходимые данные
data = [
    [
        obj['geometry']['coordinates'],
        obj['properties']['Attributes']['name'],
        obj['properties']['Attributes']['okrug']
    ]
    for obj
    in data['features']
]

In [30]:
display(set(okrug for _, _, okrug in data))

for idx, (_, _, okrug) in enumerate(data):
    if okrug == 'Южный':
        data[idx][2] = 'Южный административный округ'

display(set(okrug for _, _, okrug in data))

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный',
 'Южный административный округ'}

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный административный округ'}

Найдем округа, зайдествованные в выборке

In [53]:
# Данные по округам с википедии и гугл-карт
ref = {
        'Восточный административный округ': [37.786369, 55.796039],
        'Западный административный округ': [37.468372, 55.711506],
        'Зеленоградский административный округ': [37.194273, 55.989725],
        'Северный административный округ': [37.529975, 55.834950],
        'Северо-Восточный административный округ': [37.628449, 55.851847],
        'Северо-Западный административный округ': [37.433551, 55.821479],
        'Центральный административный округ': [37.623374, 55.753576],
        'Юго-Восточный административный округ': [37.768235, 55.689028],
        'Юго-Западный административный округ': [37.537672, 55.639672],
        'Южный административный округ': [37.661130, 55.641486]
}

ref

{'Восточный административный округ': [37.786369, 55.796039],
 'Западный административный округ': [37.468372, 55.711506],
 'Зеленоградский административный округ': [37.194273, 55.989725],
 'Северный административный округ': [37.529975, 55.83495],
 'Северо-Восточный административный округ': [37.628449, 55.851847],
 'Северо-Западный административный округ': [37.433551, 55.821479],
 'Центральный административный округ': [37.623374, 55.753576],
 'Юго-Восточный административный округ': [37.768235, 55.689028],
 'Юго-Западный административный округ': [37.537672, 55.639672],
 'Южный административный округ': [37.66113, 55.641486]}

In [47]:
# Евклидово расстояние
def distance(X, Y):
    x1, y1 = X
    x2, y2 = Y
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# Процесс работы НС Кохонена
def clusterize(Xs, Ys):
    clusters = [list() for _, name in Ys]
    for idx, X in enumerate(Xs):
        distances = [distance(X, Y) for Y, _ in Ys]
        min_idx = np.argmin(distances)
        clusters[min_idx].append((idx, X))
    return clusters

In [59]:
# Организуем данные
Xs = [point for point, _, _ in data]
Ys = [[point, name] for name, point in ref.items()]

# Кластеризация
clusters = clusterize(Xs, Ys)

# Счетчики для числа угаданных и неугаданных объектов
T, F = 0, 0
for idx, cluster in enumerate(clusters):
    # Извлекаем название округа
    _, name = Ys[idx]

    # Выводим распеределение по округам
    print(f"Site: {name}")
    c = Counter(data[jdx][2] for jdx, point in cluster)
    display(c)

    # Записываем число угаданных и неугаданных объектов
    t = c[name]
    f = sum(c.values()) - t
    T += t
    F += f

print(f"Всего: {T + F} объектов")
print(f"Верно кластеризованы: {T}")
print(f"Неверно кластеризованы: {F}")
print(f"Процент верно кластеризованных: {T/(F + T) * 100:.2f}%")
print(f"Процент неверно кластеризованных: {F/(F + T) * 100:.2f}%")

Site: Восточный административный округ


Counter({'Восточный административный округ': 31,
         'Центральный административный округ': 4})

Site: Западный административный округ


Counter({'Западный административный округ': 19})

Site: Зеленоградский административный округ


...

Site: Юго-Западный административный округ


Counter({'Юго-Западный административный округ': 12})

Site: Южный административный округ


Counter({'Южный административный округ': 20,
         'Юго-Восточный административный округ': 7,
         'Северный административный округ': 3,
         'Юго-Западный административный округ': 2})

Всего: 285 объектов
Верно кластеризованы: 229
Неверно кластеризованы: 56
Процент верно кластеризованных: 80.35%
Процент неверно кластеризованных: 19.65%
